# $\Delta G$ as a bet

Objectives of this notebook:

+ Develop statisical intuition for the meanings of $G$, $H$, and $S$. 
+ Understand the meaning of $\Delta G^{\circ \prime}$
+ Learn how to think about $\Delta G$ as a bet.


As a model for this, we are going to use a simple toy reaction (below). Imagine a table with dice on it. 

<div><center><img width="70%" src="https://github.com/harmsm/biochem-jupyter-notebooks/raw/master/dice/dice-folding-reaction.png") /></center></div>

<br/>

To get started, run the cell below.

In [ ]:
#@title Set up calculation

#@markdown This cell configures the environment for the following calculations.
#@markdown Run it before doing any of the work below.

#@markdown To run the cell, click the "Play" button to the left or hit `[Shift]+[Enter]`.

#@markdown If desired, you can run this in a notebook in a folder on your google drive. This is a 
#@markdown convenient way to pass files in and out of the analysis. It will 
#@markdown also allow you to save your work. If you put `biophysics` into the form
#@markdown field below, the analyis will save all of its calculations in the 
#@markdown `biophysics` directory in MyDrive (i.e. the top directory at
#@markdown https://drive.google.com). This script will create the directory if 
#@markdown it does not already exist. If the directory already exists, any files
#@markdown that are already in that directory will be available for the analysis. 
#@markdown You could, for example, put a file called data.csv in the biophysics
#@markdown directory and then access it as data.csv in all cells below.
#@markdown Google may prompt you for permission to access the drive. 
#@markdown <br/><br/>
#@markdown To work in a temporary environment, leave this blank. Your results
#@markdown will disappear when you leave the website.

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

# ------------------------------------------------------------------------------
# Imports

if RUNNING_IN_COLAB:
    %pip install -q ipywidgets

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy

# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
# from IPython.display import display

import os

# ------------------------------------------------------------------------------
# Environment

if RUNNING_IN_COLAB:
    
    working_dir = "/content/"

    # Select a working directory on google drive
    google_drive_directory = "" #@param {type:"string"}
    google_drive_directory = google_drive_directory.strip()

    # Set up google drive
    if google_drive_directory != "":

        from google.colab import drive
        drive.mount('/content/gdrive/')

        working_dir = f"/content/gdrive/MyDrive/{google_drive_directory}"
        os.system(f"mkdir -p {working_dir}")

    os.chdir(working_dir)
    print(f"Working directory: {os.getcwd()}/")

    print("\nCurrent directory contents:")
    print(os.getcwd())
    for f in os.listdir("."):
        print(f"    {f}")
    print()
    

# ------------------------------------------------------------------------------
# Default graph label sizing

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# For coloring output text
COL_END = '\033[0m'
COL_RED = '\033[91m'
COL_BLUE = '\033[94m'

# ------------------------------------------------------------------------------
# Notebook specific function definitions

from scipy.special import comb

def run_sim(num_dice,
            p_six=1/6,
            num_starting_six=None,
            num_steps=None,
            alpha=0.01,
            plot=False):
    """
    Run a dice simulation.
    
    Parameters
    ----------
    num_dice : int
        number of dice in the simulation (must be > 0)
    p_six : float
        probability of rolling a six (must be between 0 and 1)
    num_starting_six : int, optional
        number of six to start with. if None, select randomly. If defined, must
        be between 0 and num_dice
    num_steps : int, optional
        number of steps of the simulatino to run. if None, set to 1.5x the 
        expected time to converge
    alpha : float, defualt=0.01
        pick up this fraction of the dice and roll them each step. must be 
        between 0 and 1. (If alpha*num_dice is less than one, set the number
        to roll to one and recalcualte alpha as 1/num_dice). 
    plot : bool, default=False
        whether or not to generate a plot
        
    Returns
    -------
    steps : np.array
        array of step times 
    rate_law : np.array
        rate law calculation for the number of sixes
    dice_history : np.array
        history of the number of sixes 
    """
    
    
    # Figre out the number of steps to run if not specified
    if num_steps is None:
        
        # Biggest diff is the largest difference between the 
        # start and equilibrium that could be observed
        biggest_diff = num_dice
            
        # Figure out how may dice we roll at once
        roll_amount = alpha*num_dice
        if roll_amount < 1:
            roll_amount = 1
            
        # Expected time required to equilibrate
        if p_six > 0.125 or p_six == 0:
            roll_scalar = 0.125
        else:
            roll_scalar = p_six
        expected_time = biggest_diff/(roll_amount*roll_scalar)

        # Number of steps is 1.5*expected_time
        num_steps = int(np.around(expected_time*1.5,0))
    
    # Make sure number of steps and number of dice are sane
    if num_dice < 1 or num_steps < 1:
        err = "num_dice and num_steps must be integers > 0\n"
        raise ValueError(err)
    
    # If num_starting_six is specified, make sure it is sane
    if num_starting_six is not None:
        if num_starting_six < 0:
            err = "num_starting_six must be None or >= 0\n"
            raise ValueError(err)
    
        if num_dice < num_starting_six:
            err = "num_starting_six must be <= num_dice\n"
            raise ValueError(err)
        
    # Make sure alpha is sane
    if alpha < 0 or alpha > 1: 
        err = "alpha must be between 0 and 1\n"
        raise ValueError(err)

    # Make sure p_six is sane
    if p_six < 0 or p_six > 1: 
        err = "p_six must be between 0 and 1\n"
        raise ValueError(err)
        
    # Figure out how many dice to flip at each step
    num_to_flip = int(round(alpha*num_dice,0))
    if num_to_flip == 0:
        num_to_flip = 1
        alpha = num_to_flip/num_dice

    # Figure out the weight on our dice, assuming that 
    # p_1 == p_2 == p_3 == p_4 == p_5.  
    # p_six + Sum(p_1 through p_5) = 1 
    p_not_six = 1 - p_six
    p_each_nonsix = p_not_six/5
    dice_weights = [p_each_nonsix,p_each_nonsix,
                    p_each_nonsix,p_each_nonsix,
                    p_each_nonsix,p_six]   
    
    # Randomly select six numbers
    if num_starting_six is None:
        current_dice = np.random.choice([1,2,3,4,5,6],num_dice,p=dice_weights)
        
    # Create vector of dice
    else:    
    
        # Create non-sixes
        num_non_six = num_dice - num_starting_six
        current_dice = np.random.choice([1,2,3,4,5],num_non_six)
        
        # Extend with sixes
        current_dice = list(current_dice)
        current_dice.extend([6 for _ in range(num_starting_six)])
        current_dice = np.array(current_dice)
        
        # Shuffle dice without flipping
        np.random.shuffle(current_dice)
    
    # Create vector of step number (0 to num_steps - 1)
    steps = np.arange(num_steps)
    
    # Calculate rate law
    rate_law = p_six*num_dice*(1-np.exp(-alpha*steps)) + np.sum(current_dice==6)*np.exp(-alpha*steps)

    # Roll the dice num_steps times
    dice_history = [np.sum(current_dice == 6)]
    for i in range(1,num_steps):

        # Figure out dice to flip
        dice_to_flip = np.random.choice(np.arange(num_dice,dtype=int),num_to_flip,replace=False)
        
        # Roll num_to_flip dice
        new_dice = np.random.choice([1,2,3,4,5,6],num_to_flip,p=dice_weights)
        
        # Assign those values to the current dice
        current_dice[dice_to_flip] = new_dice
        
        # Record the history
        dice_history.append(np.sum(current_dice==6))
    
    
    if plot:
        
        # Create plots
        fig, ax = plt.subplots(1,figsize=(6,6))

        # Plot rate laws
        ax.plot(steps,rate_law,color="blue",linewidth=2,label="rate law")

        # Plot observed values
        ax.plot(steps,dice_history,"o",color="blue",label="observed")

        # Plot equilibrium line
        ax.plot((0,num_steps),(p_six*num_dice,p_six*num_dice),'k--',label="equilibrium")

        # Format plot
        ax.set_ylim(0,num_dice)
        ax.set_xlabel("time steps")
        ax.set_ylabel("number of sixes")
        ax.set_title("number of sixes over time for {} dice".format(num_dice))

        # add legend
        ax.legend()

        plt.show()

    
    return steps, rate_law, dice_history


def run_rolling_sim(num_dice,
                    p_six=1/6,
                    num_steps=None,
                    alpha=0.01):
    """
    Run dice simulations starting from equilibrium number of six, no six, and 
    all six. 
    
    Parameters
    ----------
    num_dice : int
        number of dice in the simulation (must be > 0)
    p_six : float
        probability of rolling a six (must be between 0 and 1)
    alpha : float, defualt=0.01
        pick up this fraction of the dice and roll them each step. must be 
        between 0 and 1. (If alpha*num_dice is less than one, set the number
        to roll to one and recalcualte alpha as 1/num_dice). 
        
    Returns
    -------
    fig : matplotlib.Figure
        plot figure instance
    ax : list
        list of matplotlib ax instances
    """

    # Sim with equilibrium dice
    steps, random_rate_law, num_random_sixes = run_sim(num_dice=num_dice,
                                                       p_six=p_six,
                                                       num_starting_six=None,
                                                       num_steps=num_steps,
                                                       alpha=alpha)
    
    # Sim with all six dice
    steps, start_at_six_rate_law, num_start_at_six_sixes = run_sim(num_dice=num_dice,
                                                                   p_six=p_six,
                                                                   num_starting_six=num_dice,
                                                                   num_steps=num_steps,
                                                                   alpha=alpha)
    
    # Sim with no six dice
    steps, start_no_six_rate_law, num_start_no_six_sixes = run_sim(num_dice=num_dice,
                                                                   p_six=p_six,
                                                                   num_starting_six=0,
                                                                   num_steps=num_steps,
                                                                   alpha=alpha)
        
    # Create plots
    fig, ax = plt.subplots(1,figsize=(6,6))

    # Plot rate laws
    ax.plot(steps,random_rate_law,color="blue",linewidth=2)
    ax.plot(steps,start_at_six_rate_law,color="orange",linewidth=2)
    ax.plot(steps,start_no_six_rate_law,color="green",linewidth=2)
    
    # Plot observed values
    ax.plot(steps,num_random_sixes,"o",label="start at equilibrium",color="blue")
    ax.plot(steps,num_start_at_six_sixes,"o",label="start all sixes",color="orange")
    ax.plot(steps,num_start_no_six_sixes,"o",label="start no sixes",color="green")
        
    # Plot equilibrium line
    ax.plot((0,num_steps),(p_six*num_dice,p_six*num_dice),'k--',label="p_six")
    
    # Format plot
    ax.set_ylim(0,num_dice)
    ax.set_xlabel("time steps")
    ax.set_ylabel("number of sixes")
    ax.set_title("number of sixes over time for {} dice".format(num_dice))

    # add legend
    ax.legend()

    return fig, ax


def prob_plot(p,n):
    """
    Generate a probability plot.
    
    Parameters
    ----------
    p : float
        probability of rolling a six
    n : int
        number of dice
    
    Returns
    -------
    fig, ax : matplotlib objects
    """

    fig, ax = plt.subplots(1,3,figsize=(15,5),sharex="col")

    k_range = np.arange(n+1,dtype=int)

    out1 = []
    out2 = []
    out3 = []
    for k in k_range:
        out1.append(comb(n,k))
        out2.append((p**k)*((1-p)**(n-k)))
        out3.append(out1[-1]*out2[-1])

    style = {"linestyle":"-","lw":2}
    if n <= 50:
        style["marker"] = "o"
        style["ms"] = 7    

    ax[0].plot(k_range/n,out1,**style)
    ax[1].plot(k_range/n,out2,**style)
    ax[2].plot(k_range/n,out3,**style)

    ax[0].set_title("n choose k")
    ax[1].set_title("log(likelihood)")
    ax[2].set_title("probability")

    ax[0].set_ylabel("num. cocmbinations")
    ax[1].set_ylabel("$p^{k}(1-p)^{n-k}$")
    ax[2].set_ylabel("probability")

    for i in range(3):
        ax[i].set_xlabel('fraction six')
    ax[1].set_yscale("log")

    fig.tight_layout()
    
    return fig, ax 




## <font size="+3">Probabilities</font>

<font size="+1">Why do we see predictable numbers of dice showing specific faces?</font>

In [ ]:
#@title Dice probability calculator


#@markdown The probability of seeing $k$ <font size="+2">⚅</font> out of $n$ total dice can be calculated using the following formula:

#@markdown $$P(n,k) = {n \choose k} p^{k}(1-p)^{n-k}$$

#@markdown + $n$ is the total number of dice
#@markdown + $k$ is the number of dice that are <font size="+2">⚅</font>
#@markdown + $p$ is the probability of rolling a <font size="+2">⚅</font> (i.e., $1/6$ for a fair dice)

#@markdown This cell generates graphs of the following terms for all values of $k$. 

#@markdown + $n \choose k$ : "n choose k" --> entropy
#@markdown + $log \Big (p^{k}(1 - p)^{n-k} \Big )$ : "log(likelihood)"  --> enthalpy
#@markdown + $P(n,k)$ : "probability" --> free energy

p = 1/6 #@param
n = 600 #@param {type:"integer"}

#@markdown <font color="orange" size="+1">Pay careful attention to the magnitude of the axes.</font>

_ = prob_plot(p=p,n=n)

### <font color='#9D33FF' size="+2">Explain</font>

<font size="+1">Generate graphs for $n = 1$, $p = 1/6$ (one fair dice). In words: how do $n \choose k$ and $p^{k}(1 - p)^{n-k}$ determine the probabilities of seeing 0 <font size="+2">⚅</font> versus 1 <font size="+2">⚅</font></font>?


---

### <font color='#9D33FF' size="+2">Effect of increasing $n$:</font>

<font size="+1">Generate graphs with $n = 1$, $n = 10$, and $n = 100$.</font>

<font size="+1">How does the "n choose k" graph change? Why, mathematically, is this the case?</font>

---
<font size="+1">How does the "log(likelihood)" graph change? Why, mathematically, is this the case?</font>

---

<font size="+1">How does the "probability" graph change? Can you explain this in terms of the "n choose k" and "log(likelihood)" graphs?</font>

---

### <font color='#9D33FF' size="+2">Effect of $p$:</font>

<font size="+1">Compare the graph for $n = 100$, $p = 1/6$ to the graph for $n = 100$, $p = 1/2$ (weighted dice that show <font size="+2">⚅</font> half the time). What happens to the log(likelihood) graph? Why does this happen with $p = 1/2$?</font>

---

### <font color="#4287f5" size="+3">Key Point</font>

<font size="+1">The highest probability configuration of a system (the lowest free energy) involves a tradeoff between:

1. <font size="+1">Entropy: Configurations that are probable because they are *degenerate*. This is high $n \choose k$. </font>
2. <font size="+1">Enthalpy: Configurations that consist of high probability events. This is high $p^{k}(1-p)^{k}$.</font>


**This is really is our familiar friend $G = H - TS$**. We are doing the calculations under the special case where $RT = 1$. (These are *reduced temperature units* and are used a lot in theoretical calculations). 

$$P(n,k) = p^{k}(1-p)^{n-k} {n \choose k}$$
$$-ln \Big (P(n,k) \Big) = -ln \Big ( p^{k}(1-p)^{n-k} {n \choose k} \Big )$$

$$-ln \Big(P(n,k) \Big) = -ln \Big ( p^{k}(1-p)^{n-k} \Big ) - ln \Big ( {n \choose k} \Big )$$

$$G \equiv -ln \Big ( P(n,k) \Big )$$
$$H \equiv -ln \Big ( p^{k}(1-p)^{n-k} \Big )$$
$$S \equiv ln \Big ( {n \choose k} \Big )$$

$$G = H - S$$



</font>

## <font size="+3">The Dice Reaction</font>

Even dice on a table will equilibrate if you start rolling them.

In [ ]:
#@title Equilibrium simulation

#@markdown In this calculation, we have 600 dice with $p = 1/6$. We then
#@markdown pick up and roll a dice 10,000 times.

NUM_DICE = 600
P_SIX = 1/6

num_starting_six = int(np.round(NUM_DICE*P_SIX,0))
_, _, obs = run_sim(NUM_DICE,
                    p_six=P_SIX,
                    num_starting_six=num_starting_six,
                    num_steps=10000,
                    alpha=1/NUM_DICE,
                    plot=True)

num_bins = 100 #np.sqrt(NUM_DICE)
if num_bins > NUM_DICE:
    num_bins = NUM_DICE


counts, bins = np.histogram(obs,
                            bins=np.linspace(0,NUM_DICE+1,num_bins+1))


counts = counts/np.sum(counts)
bins_as_fraction = bins[0:-1] + (bins[1]-bins[0])/2
bins_as_fraction = bins_as_fraction/np.max(bins_as_fraction)


fig, ax  = plt.subplots(1,figsize=(6,6))

_ = ax.plot(bins_as_fraction,counts,lw=2,color="blue")
ax.set_xlabel("fraction six")
ax.set_ylabel("frequency")
ax.set_title("observed fraction of six")
plt.show()


<h2><font color='#9D33FF'>Observe</font></h2>

<font size="+1">How does the number of sixes change over time? How does this empirical distribution compare to the one we calculated analytically above?</font>

---

<h2><font color='#9D33FF'>Make a prediction</font></h2>

<font size="+1">What would the graph look like if we started with all 600 dice showing <font size="+2">⚅</font>? Why will the graph have this shape?</font>


----


<font size="+1">Run the "Out of equilibrium simulation" cell below. Did thje results match your prediction? If not, can you figure out why?</font>

---

In [ ]:
#@title Out of equilibrium simulation

#@markdown In this calculation, we have NUM_DICE dice. The probability of rolling
#@markdown <font size="+2">⚅</font> is P_SIX. We nave NUM_STARTING_SIX <font size="+2">⚅</font>
#@markdown at the beginning of the simulation. The simulation runs until it 
#@markdown equilibrates. 

NUM_DICE = 600                  #@param {type:"integer"}
P_SIX = 1/6     #@param 
NUM_STARTING_SIX = 600 #@param {type:"integer"}

_ = run_sim(NUM_DICE,
            p_six=P_SIX,
            num_starting_six=NUM_STARTING_SIX,
            alpha=1/NUM_DICE,
            plot=True)

<h2><font color='#9D33FF'>Make a prediction</font></h2>

<font size="+1">What would the graph look like if we started with no dice showing <font size="+2">⚅</font>? Why will the graph have this shape?</font>


---

<h2><font color='#9D33FF'>Make a prediction</font></h2>

<font size="+1">What would the graph look like if we started with 300 of the 600  dice showing <font size="+2">⚅</font>? Why will the graph have this shape?</font>


---

<h2><font color='#9D33FF'>Standard state free energy?</font></h2>

<font size="+1">On the graph where we started with 300 of the 600 dice showing <font size="+2">⚅</font>, where would you find the following terms?</font>

$$G_{eq}$$
$$G^{\circ \prime}$$
$$\Delta G^{\circ \prime}$$

What does the value of $\Delta G^{\circ \prime} measure?$?


---

### <font color="#4287f5" size="+3">Key Point</font>


<font size="+1">Reactions spontaneously move from low-probability, out-of-equilibrium configurations to the highest probability (equilibrium) configuration</font>

<font size="+1">$\Delta G^{\circ \prime}$ is the energy to go from standard state (1 M everything) to equilibrium.</font>

## <font size="+3">The Bet</font>

Uncle Phil says he'll give you $1,000 if you guess the next transition you will see: 

<center>
<font size="+2">{⚀,⚁,⚂,⚃,⚄}</font> $\rightarrow$ <font size="+2">⚅</font>

OR

<font size="+2">⚅</font>$\rightarrow$ <font size="+2">{⚀,⚁,⚂,⚃,⚄}</font>


(If you roll and do not see a change (e.g., <font size="+2">⚅</font> $\rightarrow$ <font size="+2">⚅</font>), you re-roll.)

</center>

In [ ]:
#@title Simulator

#@markdown Simulates betting on a table with 600 dice. It will run 10 times. The
#@markdown table resets each time it runs. 

NUM_STARTING_SIX = 100 #@param
BETTING_STRATEGY = "bet on 6 -> {1,2,3,4,5}" #@param ["bet on {1,2,3,4,5} -> 6", "bet on 6 -> {1,2,3,4,5}"]

for i in range(10):

    _, _, obs = run_sim(600,num_starting_six=NUM_STARTING_SIX,alpha=1/600,num_steps=100)
    obs = np.array(obs)
    from_six = obs[:-1] > obs[1:]
    to_six = obs[:-1] < obs[1:]
    no_change = obs[:-1] == obs[1:]

    result = None
    for i in range(len(from_six)):
        if from_six[i]:
            result = "from_six"
            break

        if to_six[i]:
            result = "to_six"
            break


    if result == "from_six":
        if BETTING_STRATEGY == "bet on 6 -> {1,2,3,4,5}":
            success = True
        else:
            success = False

    elif result == "to_six":
        if BETTING_STRATEGY == "bet on {1,2,3,4,5} -> 6":
            success = True
        else:
            success = False

    else:
        raise RuntimeError("Should not get here")

    if success:
        print(f"You just {COL_BLUE}made{COL_END} $1000!")
    else:
        print( f"You just {COL_RED}lost{COL_END}.")



You have 600 dice on the table, 100 of which are <font size="+2">⚅</font>. 

<h2><font color='#9D33FF'>Make a prediction</font></h2>
<font size="+1">What batting strategy should you choose? What are the odds you get $1,000?</font>

---

If you could choose a value for the number of <font size="+2">⚅</font> to maximize you chances of getting the money, what would you choose? If you run the simulation, does the strategy seem to work? 


----

Uncle Phil changes the deal. He tells you he will give you \$1 for every right guess and will *take away* \$1 for every wrong guess. As before, you are guessing which event you see. 

<center>
<font size="+2">{⚀,⚁,⚂,⚃,⚄}</font> $\rightarrow$ <font size="+2">⚅</font>

OR

<font size="+2">⚅</font>$\rightarrow$ <font size="+2">{⚀,⚁,⚂,⚃,⚄}</font>


(If you roll and do not see a change (e.g., <font size="+2">⚅</font> $\rightarrow$ <font size="+2">⚅</font>), you re-roll.)

</center>


In [ ]:
#@title Simulator

#@markdown Simulates betting on a table with 600 dice. 

NUM_STARTING_SIX = 600 #@param
BETTING_STRATEGY = "bet on 6 -> {1,2,3,4,5}" #@param ["bet on {1,2,3,4,5} -> 6", "bet on 6 -> {1,2,3,4,5}", "random bet each roll"]



_, _, obs = run_sim(600,num_starting_six=NUM_STARTING_SIX,alpha=1/600)
obs = np.array(obs)
from_six = obs[:-1] > obs[1:]
to_six = obs[:-1] < obs[1:]
no_change = obs[:-1] == obs[1:]

if BETTING_STRATEGY == "bet on {1,2,3,4,5} -> 6":
    cash = np.sum(to_six) - np.sum(from_six)
elif BETTING_STRATEGY == "bet on 6 -> {1,2,3,4,5}":
    cash = np.sum(from_six) - np.sum(to_six)
else:
    bets = np.random.choice([False,True],len(from_six))
    mask = np.logical_not(no_change)

    right = np.logical_and(bets[mask],to_six[mask])
    wrong = np.logical_and(np.logical_not(bets[mask]),from_six[mask])
    
    cash = np.sum(right) - np.sum(wrong)


if cash < 0:
    print( f"You just {COL_RED}lost{COL_END} ${-cash}!")
elif cash > 0:
    print(f"You just {COL_BLUE}made{COL_END} ${cash}!")
else:
    print(f"You broke even.")

<h2><font color='#9D33FF'>Make a prediction</font></h2>

<font size="+1">The table starts with 600 <font size="+2">⚅</font> of out 600 dice. You will roll until the reaction has equlibrated. What should your betting strategy be? How much money do you expect to make or lose? Why?</font>

---

<font size="+1">How did you do when you tried? What did you change (if anything) and why?</font>

In [ ]:
#@title dG versus cash made

#@markdown This cell does the last betting run for different starting amounts of
#@markdown <font size="+2">⚅</font>: 0, 100, 200, 300, 400, 500, and 600. The 
#@markdown total number of dice is 600. It then correlates the cash made in the
#@markdown betting with the $\Delta G$ calculated by:

#@markdown $$H \equiv -ln \Big ( p^{k}(1-p)^{n-k} \Big )$$
#@markdown $$S \equiv ln \Big ( {n \choose k} \Big )$$
#@markdown $$G = H - S$$

#@markdown $$\Delta G = G_{eq} - G_{starting_six}$$

S = np.log(comb(600,100))
H = -100*np.log(1/6) + -500*np.log(5/6)
Geq = H - S

mean_cash = []
std_cash = []
dG = []

print("num_six_at_start,mean_cash_made,dG")
for N in [0,100,200,300,400,500,600]:

    cash_made = []
    for i in range(10):

        _, _, obs = run_sim(600,num_starting_six=N,alpha=1/600,num_steps=5000)
        obs = np.array(obs)
        from_six = obs[:-1] > obs[1:]
        to_six = obs[:-1] < obs[1:]
        no_change = obs[:-1] == obs[1:]

        cash = np.sum(from_six) - np.sum(to_six)

        cash_made.append(cash)

    mean_cash.append(np.mean(cash_made))
    std_cash.append(np.std(cash_made))


    S = np.log(comb(600,N))
    H = -N*np.log(1/6) + -(600-N)*np.log(5/6)
    G = H - S
    dG.append(Geq-G)

    print(N,mean_cash[-1],dG[-1])

fig, ax = plt.subplots(1,figsize=(6,5))

ax.plot(mean_cash,dG,'o')
ax.set_xlabel("cash made")
ax.set_ylabel("dG")
fig.tight_layout()

---

### <font color="#4287f5" size="+3">Key Point</font>

The further away from equilibrium a system is, the better you will be at making money. Free energy measures your betting odds: the larger the magnitude of $\Delta G$, the more confident you can be that the reaction will flow one way or the other. Cells keep $\Delta G \ne 0$ to allow ordered and directional behavior. 
